## astropy.wcs
Implements the FITS WCS standard and some commonly used distortion conventions.

This tutorial will show how to create a WCS object from a FITS file and how to use it to transform coordinates.

In [1]:
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
import os
from astropy.io import fits
from astropy import wcs

Open a file with `astropy.fits` and look at it.

In [3]:
sip_file_name = os.path.join('data/sip.fits')

sip_file = fits.open(sip_file_name)
sip_file.info()

Filename: data/sip.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      62   ()      


To create a WCS object pass the header with the WCS kyewords to astropy.wcs.WCS (Primary header in this case).

In [4]:
w = wcs.WCS(sip_file[0].header)

To perform the transformation from detector to sky, including distortions, pass x and y and an 'origin'. The third argument, 'origin', indicates whether the coordinates are 1-based (like FITS), or 0-based (like python).

The inputs can be numbers, numpy arrays or array like objects.

In [5]:
ra, dec = w.all_pix2world([1, 1], [2, 2], 1)
print(ra, dec)

[202.39347618 202.39347618] [47.1772851 47.1772851]


Perfom the inverse transformation - from sky to detector coordinates.

If analytical inverse is not available (often the case in the presence of distortion), then an iterative inverse is performed.

In [6]:
print(w.all_world2pix(ra, dec, 1))

[array([1.00000005, 1.00000005]), array([2.00000006, 2.00000006])]


In some cases it is useful to omit the distortion and perform the core WCS transforms only:

In [7]:
ra, dec = w.wcs_pix2world([1, 1], [2, 2], 1)
print(ra, dec)

[202.39299121 202.39299121] [47.17731548 47.17731548]


In [8]:
w.wcs_world2pix(ra, dec, 1)

[array([1., 1.]), array([2., 2.])]

The WCS object can be changed and the new WCS can be written out as a new header.

By default only the primary WCS keywords are written out to the header. Pass a keyword `relax=True` to write out the SIP distortion.

In [9]:
# The original WCS
w.printwcs()

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN-SIP'  'DEC--TAN-SIP'  
CRVAL : 202.482322805  47.17511893  
CRPIX : 128.0  128.0  
PC1_1 PC1_2  : 0.000249756880272  0.000230177809744  
PC2_1 PC2_2  : 0.000230428519265  -0.000249965770577  
CDELT : 1.0  1.0  
NAXIS : 0  0


In [10]:
w.wcs.crpix = [200, 200]

# Calling *to_header()* without arguments writes
# out the standard WCS keywords.
#w.to_header()

# Passing *relax=True* writes out the SIP coefficients
# and all other distortions.
w.to_header(relax=True)

WCSAXES =                    2 / Number of coordinate axes                      
CRPIX1  =                200.0 / Pixel coordinate of reference point            
CRPIX2  =                200.0 / Pixel coordinate of reference point            
PC1_1   =    0.000249756880272 / Coordinate transformation matrix element       
PC1_2   =    0.000230177809744 / Coordinate transformation matrix element       
PC2_1   =    0.000230428519265 / Coordinate transformation matrix element       
PC2_2   =   -0.000249965770577 / Coordinate transformation matrix element       
CDELT1  =                  1.0 / [deg] Coordinate increment at reference point  
CDELT2  =                  1.0 / [deg] Coordinate increment at reference point  
CUNIT1  = 'deg'                / Units of coordinate increment and value        
CUNIT2  = 'deg'                / Units of coordinate increment and value        
CTYPE1  = 'RA---TAN-SIP'       / TAN (gnomonic) projection + SIP distortions    
CTYPE2  = 'DEC--TAN-SIP'    

### TPV distortion

The TPV distortion convention understood by SExtractor and related programs is also understood by `astropy.wcs`.

A header can be read from a text file and used to create the WCS. Here we use a modified header from the Palomar Transient Factory.

In [11]:
tpv_header = fits.Header.fromtextfile(os.path.join('data', 'PTF_r_chip01_tpv.txt'))

In [12]:
w_tpv = wcs.WCS(tpv_header)

The keywords starting with `PV` specify the distortion

In [13]:
w_tpv.to_header()

WCSAXES =                    2 / Number of coordinate axes                      
CRPIX1  =             931.4572 / Pixel coordinate of reference point            
CRPIX2  =             2344.964 / Pixel coordinate of reference point            
PC1_1   =  0.00028091397897264 / Coordinate transformation matrix element       
PC1_2   =  3.2908700558791E-06 / Coordinate transformation matrix element       
PC2_1   =  3.4654449727329E-06 / Coordinate transformation matrix element       
PC2_2   = -0.00028106139503453 / Coordinate transformation matrix element       
CDELT1  =                  1.0 / [deg] Coordinate increment at reference point  
CDELT2  =                  1.0 / [deg] Coordinate increment at reference point  
CUNIT1  = 'deg'                / Units of coordinate increment and value        
CUNIT2  = 'deg'                / Units of coordinate increment and value        
CTYPE1  = 'RA---TPV'           / TAN (gnomonic) projection + distortions        
CTYPE2  = 'DEC--TPV'        

In the TPV case, `all_pix2world` gives the same result as `wcs_pix2world`.

This is because the underlying WCSLIB library is handling the distortion.

In [14]:
print(w_tpv.all_pix2world([[500,1000]], 1))

[[68.94746342 26.71561128]]


In [15]:
print(w_tpv.wcs_pix2world([[500,1000]], 1))

[[68.94746342 26.71561128]]


### Exercise 1:

- Create a WCS object from the file. 

`dist_file_name = 'data/dist_lookup.fits.gz'`

This file contains all distortions typical for HST imaging data - SIP, lookup_table and det2im (detector to image - correcting detector irregularities). The lookup table and det2im distortions are stored in separate extensions so you will need to pass as a second argument to `wcs.WCS` the file object (already opened with astropy.io.fits).

- Look at the file object with the `info()` method. The lookup_table and det2im distortions are saved in separate extensions.

- Modify one of the WCS keywords and save it to file. (As some of the distortion is saved in extensions, use the method `to_fits()` to save the entire WCS.

### Exercise 2:

The FITS WCS standard supports alternate WCSs in the same eader.
These are defined by the same keywords with a character (A...Z) appended
to them. For example, *CRPIXA1*, etc.

Using the same file create a WCS object for the alternate WCS in this header, by passing also `key='O'` to wcs.WCS().
Compare the two WCSs using the `printwcs()` method`

In [22]:
# Open FITS files to play with
dist_file_name = 'data/dist_lookup.fits.gz'
f = fits.open(dist_file_name)

# Print extensions since "The lookup_table 
# and det2im distortions are saved in separate 
# extensions."
f.info()

# Read the WCS information in header
# Note the second argument to WCS is the file object handle, which the WCS docs note
# "is needed when header keywords point to a distortion paper lookup table stored 
# in a different extension." (also noted above)
w_test = wcs.WCS(f[1].header, f)
# Print current header
w_test.printwcs()

# Modify header value for reference pixel
w_test.wcs.crpix = [200, 200]  # Just trying to change the header value

# Apparently this will not work because the WCS information is in several extensions
#hdr_test = w_test.to_header(relax=True)
# new_f.printwcs()

# Create new FITS file in memory
new_f = w_test.to_fits(relax=True)
new_f.info()
new_f[0].header['CRPIX*']

Filename: data/dist_lookup.fits.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  SCI           1 ImageHDU       170   (100, 100)   float32   
  2  D2IMARR       1 ImageHDU        15   (4096, 1)   float32   
  3  WCSDVARR      1 ImageHDU        15   (65, 33)   float32   
  4  WCSDVARR      2 ImageHDU        15   (65, 33)   float32   
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN-SIP'  'DEC--TAN-SIP'  
CRVAL : 5.63056810618  -72.05457184278998  
CRPIX : 2048.0  1024.0  
CD1_1 CD1_2  : 1.29055156973602e-05  5.9525007864732e-06  
CD2_1 CD2_2  : 5.02263821027651e-06  -1.2644844123757e-05  
NAXIS : 100  100
Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      65   ()      
  1  D2IMARR       1 ImageHDU        15   (4096, 1)   float32   
  2  WCSDVARR      1 ImageHDU        15   (65, 33)   float32   
  3  WCSDVARR      2

CRPIX1  =                200.0 / Pixel coordinate of reference point            
CRPIX2  =                200.0 / Pixel coordinate of reference point            

In [21]:
# Read the WCS information in header (from the open FITS file)
w_test = wcs.WCS(f[1].header, f)
w_testO = wcs.WCS(f[1].header, f, key='O')

# Print the two headers
w_test.printwcs()
w_testO.printwcs()

KeyError: "No WCS with key 'O' was found in the given header"